<a href="https://colab.research.google.com/github/sibat119/papers-review-code-impl/blob/main/lda/lda_with_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importing the Required Libraries**


In [1]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 175 kB/s 
     |████████████████████████████████| 11.3 MB 51.6 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=7ba082316e24d04c1170adfd49f0f98e1b728924130415271d7a3a401f0345df
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [1]:
!pip install nltk
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
import numpy as np
import pandas as pd
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#vis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


## **Load Data**

In [5]:
data = pd.read_csv("/content/Review.csv")
data.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text
0,0,226241,B002T0NWKE,A1G25CG7UWQ3XO,&#10003;Onoñda&#660;gegá&#660;,1,5,1,1309305600,Misleading discription-bad taste,they advertise that they contain aspartame die...,43
1,1,194272,B0009Y8AGI,A34Y9SS1F0H485,Bluemoon,0,6,1,1263600000,Yuck!,taste disgusting having struggle just keep dow...,27
2,2,359016,B003CIBPN8,ANDTRFIVWED40,R. Hauff,6,8,1,1275782400,"If you liked it in tetrapak box, don't buy now",bought this zico water hoping have good taste ...,53
3,3,402750,B000EQX6HO,A2W95LYEAIRDAC,"I. B. Lehman ""IBL""",1,1,1,1316044800,Great product...just not in small packages!,love terra chipswhen they bags from storei ord...,49
4,4,429211,B0006VM0LU,A32T7H92YWC0A2,Molly F,0,0,1,1317340800,Unfortunately unsalvageable,wanted desperately like this sauce smells tast...,35


In [4]:
stopwords = stopwords.words("english")

In [6]:
print(len(stopwords), stopwords)

179 ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

In [7]:
reviews = pd.DataFrame(data, columns= ['Text'])
print (reviews)

                                                   Text
0     they advertise that they contain aspartame die...
1     taste disgusting having struggle just keep dow...
2     bought this zico water hoping have good taste ...
3     love terra chipswhen they bags from storei ord...
4     wanted desperately like this sauce smells tast...
...                                                 ...
9995  rescued cats female urinary infectionsshe very...
9996  love zipfizz quick energy like shakey caffeine...
9997  much except that this tastes same campbells to...
9998  after having craving mint several days went lo...
9999  office hooked these ordered assorted case from...

[10000 rows x 1 columns]


In [8]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

# df = pd.DataFrame(['this was cheesy', 'she likes these books', 'wow this is great'], columns=['text'])
reviews['text_lemmatized'] = reviews.Text.apply(lemmatize_text)
print(reviews)

                                                   Text                                    text_lemmatized
0     they advertise that they contain aspartame die...  [they, advertise, that, they, contain, asparta...
1     taste disgusting having struggle just keep dow...  [taste, disgusting, having, struggle, just, ke...
2     bought this zico water hoping have good taste ...  [bought, this, zico, water, hoping, have, good...
3     love terra chipswhen they bags from storei ord...  [love, terra, chipswhen, they, bag, from, stor...
4     wanted desperately like this sauce smells tast...  [wanted, desperately, like, this, sauce, smell...
...                                                 ...                                                ...
9995  rescued cats female urinary infectionsshe very...  [rescued, cat, female, urinary, infectionsshe,...
9996  love zipfizz quick energy like shakey caffeine...  [love, zipfizz, quick, energy, like, shakey, c...
9997  much except that this tastes sa

In [9]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        if len(new) > 0:
          final.append(new[0])
    return(final)

data_words = gen_words(reviews)
reviews['words'] = reviews.text_lemmatized.apply(gen_words)

In [10]:
print(reviews['words'][0][:10])

['they', 'advertise', 'that', 'they', 'contain', 'aspartame', 'diet', 'flavor', 'boldly', 'state']


In [11]:
id2word = corpora.Dictionary(reviews['words'])

corpus = []
for words in reviews['words']:
  new = id2word.doc2bow(words)
  corpus.append(new)


In [12]:
print(corpus[0][:20])
word = id2word[[0][:1][0]]
print(word)

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 3), (11, 1), (12, 2), (13, 1), (14, 3), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2)]
advertise


In [13]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.05,
                                            eta=0.05)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

## **Vizualizing the Data**

In [14]:
pyLDAvis.enable_notebook()

In [15]:
# lda_viz = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis_data = gensimvis.prepare(lda_model, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [16]:
pyLDAvis.display(vis_data)

## **Find optimum topic count**

In [ ]:

lda_5 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.05,
                                            eta=0.05)


lda_15 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.05,
                                            eta=0.05)


lda_25 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=25,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.05,
                                            eta=0.05)


lda_50 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.05,
                                            eta=0.05)

In [18]:
vis_data_5 = gensimvis.prepare(lda_5, corpus, id2word)
pyLDAvis.display(vis_data_5)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [19]:
vis_data_15 = gensimvis.prepare(lda_15, corpus, id2word)
pyLDAvis.display(vis_data_15)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [20]:
vis_data_25 = gensimvis.prepare(lda_25, corpus, id2word)
pyLDAvis.display(vis_data_25)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [21]:
vis_data_50 = gensimvis.prepare(lda_50, corpus, id2word)
pyLDAvis.display(vis_data_50)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


## **vary alpha and eta with topic number is 15**

In [23]:
lda_alpha_5 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.005,
                                            eta=0.05)

lda_alpha_10 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.10,
                                            eta=0.05)

lda_alpha_15 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.15,
                                            eta=0.05)

lda_alpha_20 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.20,
                                            eta=0.05)

lda_alpha_25 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.25,
                                            eta=0.05)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [24]:
vis_data = gensimvis.prepare(lda_alpha_5, corpus, id2word)
pyLDAvis.display(vis_data)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [1]:
vis_data = gensimvis.prepare(lda_alpha_10, corpus, id2word)
pyLDAvis.display(vis_data)

NameError: ignored

In [ ]:
vis_data = gensimvis.prepare(lda_alpha_15, corpus, id2word)
pyLDAvis.display(vis_data)

In [ ]:
vis_data = gensimvis.prepare(lda_alpha_20, corpus, id2word)
pyLDAvis.display(vis_data)

In [ ]:
vis_data = gensimvis.prepare(lda_alpha_25, corpus, id2word)
pyLDAvis.display(vis_data)